In [2]:
import  openpyxl 
import pandas as pd
import numpy as np
import os
import re

In [3]:
def checkVersion(workbook):
        cell_value = workbook['Info'].cell(row=4, column=4).value
        match = re.search(r"Version:\s*([\d.]+)\s*as", cell_value or "")
        version = float(match.group(1)[:3]) if match else None
        return version

In [4]:
def get_topleft_of_merged_cells(sheet, coordinate):
    for merged_range in sheet.merged_cells.ranges:
        if coordinate in merged_range:
            return merged_range.coord.split(":")[0]
    return coordinate


In [5]:
def get_data_from_ranges(sheet, ranges):
    data = []
    for cell_range in ranges:
        if ":" not in cell_range:
            cell = sheet[cell_range]
            data.append({'Cell': cell.coordinate, 'Column2': cell.value, 'Count_values': 1 if cell.value is not None else 0})
        else:
            for row in sheet[cell_range]:
                for cell in row:
                    data.append({'Cell': cell.coordinate, 'Column2': cell.value, 'Count_values': 1 if cell.value is not None else 0})
    return data

In [6]:
def get_data_from_ranges(sheet, ranges):
    data = []
    for cell_range in ranges:
        if ":" not in cell_range:
            cell = sheet[cell_range]
            top_left = get_topleft_of_merged_cells(sheet, cell.coordinate)
            actual_value = sheet[top_left].value
            data.append({'Cell': cell.coordinate, 'Column2': actual_value, 'Count_values': 1 if actual_value is not None else 0})
        else:
            for row in sheet[cell_range]:
                for cell in row:
                    top_left = get_topleft_of_merged_cells(sheet, cell.coordinate)
                    actual_value = sheet[top_left].value
                    data.append({'Cell': cell.coordinate, 'Column2': actual_value, 'Count_values': 1 if actual_value is not None else 0})
    return data

In [7]:
def get_ranges_for_version(version):
    if version in [2.4,2.5]:
        return ['E7:E25','E27','E30:F35','E39:F48','F59','C63','F63','J63:J72','E75:I83']
    elif version in [2.6] :
        return ['E7:E25','E27','E30:F35','E39:F48','F59','C63','F63','J63:J73','E76:I83']
    else:
        return [] 

In [8]:
def process_excel_files(folder_path, file_names, sheet_name):
    summed_data = {}
    cell_file_count = {}

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        workbook = openpyxl.load_workbook(file_path, data_only=True)

        version = checkVersion(workbook)
        if not version or version < 2.4 or sheet_name not in workbook.sheetnames:
            continue

        sheet = workbook[sheet_name]
        range_ver = get_ranges_for_version(version)
        data = get_data_from_ranges(sheet, range_ver)

        df = pd.DataFrame(data)
        for _, row in df.iterrows():
            key = row['Cell']
            if key not in summed_data:
                summed_data[key] = {'Column2': row['Column2'], 'Count_values': 0}
                cell_file_count[key] = 0

            summed_data[key]['Count_values'] += row['Count_values']
            cell_file_count[key] += 1

    result_data = {}
    for cell, values in summed_data.items():
        frequency = values['Count_values'] / cell_file_count[cell] if cell_file_count[cell] > 0 else 0
        result_data[cell] = round(frequency, 2)

    return result_data

In [11]:
folder_path = os.path.join(os.getcwd(), "TEST_for_Measurement_Equipment")
sheet_name = 'Warranty_Sys'
#file_names = [f for f in os.listdir(folder_path) if f.endswith('.xlsm') and 'PPA' in f.upper()]
file_names = [f for f in os.listdir(folder_path)]
result_data = process_excel_files(folder_path, file_names, sheet_name)

In [10]:
result_sheet = openpyxl.load_workbook('My_version_template.xlsx', keep_vba= False)
ws = result_sheet['Warranty_Sys'] 

def get_topleft_of_merged_cells(sheet, coordinate):
    for merged_range in sheet.merged_cells.ranges:
        if coordinate in merged_range:
            return merged_range.coord.split(":")[0]
    return coordinate

# Write data back to the sheet
for key, value in result_data.items():
    top_left = get_topleft_of_merged_cells(ws, key) 
    ws[top_left] = value

result_sheet.save('My_version_template_saved.xlsx')